# Augmentation Techniques from Scratch

In [6]:
import os
from PIL import Image
import numpy as np
# Put your imports here

## Defining the Techniques

### Put the name of the augmentation sheet here as a title

In [7]:
# Create a function that implements the augmentation technique in a code section like this
# The function must be dynamic i.e. it can have multiple differnt values to augment

### Saturation

In [7]:
def rgb_to_hsl(r, g, b):
    r, g, b = r / 255.0, g / 255.0, b / 255.0
    max_c = np.maximum(np.maximum(r, g), b)
    min_c = np.minimum(np.minimum(r, g), b)
    lightness = (max_c + min_c) / 2.0

    saturation = np.zeros_like(lightness)
    hue = np.zeros_like(lightness)

    mask = max_c != min_c
    dif = max_c - min_c
    saturation[mask] = np.where(lightness[mask] > 0.5, dif[mask] / (2.0 - max_c[mask] - min_c[mask]), dif[mask] / (max_c[mask] + min_c[mask]))

    mask_r = (max_c == r) & mask
    mask_g = (max_c == g) & mask
    mask_b = (max_c == b) & mask

    hue[mask_r] = ((g[mask_r] - b[mask_r]) / dif[mask_r] + (g[mask_r] < b[mask_r]) * 6) % 6
    hue[mask_g] = ((b[mask_g] - r[mask_g]) / dif[mask_g] + 2) % 6
    hue[mask_b] = ((r[mask_b] - g[mask_b]) / dif[mask_b] + 4) % 6
    hue /= 6

    return hue, saturation, lightness

def hsl_to_rgb(hue, saturation, lightness):
    def hue_to_rgb(p, q, t):
        t = np.where(t < 0, t + 1, t)
        t = np.where(t > 1, t - 1, t)
        return np.where(t < 1 / 6, p + (q - p) * 6 * t, 
                        np.where(t < 1 / 2, q,
                                 np.where(t < 2 / 3, p + (q - p) * (2 / 3 - t) * 6, p)))

    q = np.where(lightness < 0.5, lightness * (1 + saturation), lightness + saturation - lightness * saturation)
    p = 2 * lightness - q
    r = hue_to_rgb(p, q, hue + 1 / 3)
    g = hue_to_rgb(p, q, hue)
    b = hue_to_rgb(p, q, hue - 1 / 3)
    return (r * 255).astype(np.uint8), (g * 255).astype(np.uint8), (b * 255).astype(np.uint8)

def adjust_saturation(image, saturation_factor):
    r, g, b = image[..., 0], image[..., 1], image[..., 2]
    hue, saturation, lightness = rgb_to_hsl(r, g, b)
    saturation *= saturation_factor
    saturation = np.clip(saturation, 0, 1)

    r, g, b = hsl_to_rgb(hue, saturation, lightness)
    new_pixels = np.stack([r, g, b], axis=-1)

    new_image = Image.fromarray(new_pixels, 'RGB')
    return new_image

### Flipping

In [10]:
def flip_image(image, flip_type='horizontal'):
    if flip_type == 'horizontal':
        flipped_pixels = np.fliplr(image)
    elif flip_type == 'vertical':
        flipped_pixels = np.flipud(image)
    else:
        raise ValueError("Invalid flip_type. Choose from 'horizontal' or 'vertical'.")
    
    flipped_image = Image.fromarray(flipped_pixels)
    return flipped_image

### Brightness

In [8]:
def adjust_brightness(image, brightness_factor):
    r, g, b = image[..., 0], image[..., 1], image[..., 2]
    hue, saturation, lightness = rgb_to_hsl(r, g, b)
    lightness *= brightness_factor
    lightness = np.clip(lightness, 0, 1)

    r, g, b = hsl_to_rgb(hue, saturation, lightness)
    new_pixels = np.stack([r, g, b], axis=-1)

    new_image = Image.fromarray(new_pixels, 'RGB')
    return new_image

### Rotation

In [9]:
def rotate_image(image, angle):
    pil_image = Image.fromarray(image)
    rotated_pil_image = pil_image.rotate(angle, expand=True)
    rotated_pixels = np.array(rotated_pil_image)
    rotated_image = Image.fromarray(rotated_pixels)
    return rotated_image

## Applying the transformations

In [11]:
def save_image(image, original_name, suffix, output_dir):
    base_name, ext = os.path.splitext(original_name)
    new_name = f"{base_name}_{suffix}{ext}"
    image.save(os.path.join(output_dir, new_name))

input_dir = '../images'
output_dir = 'augmented_images'

image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpeg'))]

for image_file in image_files:
    image_path = os.path.join(input_dir, image_file)
    image = Image.open(image_path)
    pixels = np.array(image)
    transformed_images = {}
    
    # Transformations
    transformed_images['saturation_0.5'] = adjust_saturation(pixels, 0.5)
    transformed_images['saturation_1.5'] = adjust_saturation(pixels, 1.5)
    transformed_images['flipped_horizontal'] = flip_image(pixels, 'horizontal')
    transformed_images['flipped_vertical'] = flip_image(pixels, 'vertical')
    transformed_images['brightness_0.5'] = adjust_brightness(pixels, 0.5)
    transformed_images['brightness_1.5'] = adjust_brightness(pixels, 1.5)
    transformed_images['rotated_90'] = rotate_image(pixels, 90)
    transformed_images['rotated_180'] = rotate_image(pixels, 180)

    for suffix, img in transformed_images.items():
        save_image(img, image_file, suffix, output_dir)